# 🧠 Product Info Extractor using LangChain + LLM

### 🎯 Objective: Build a simple assistant using any LLM (e.g., OpenAI, Groq) that extracts structured product information using a Pydantic schema.

### ❓ Task Description: When asked about a product, your assistant should return a structured response with the following fields:

- **🧾 Product Name**: A one-word name of the product (from a textile company or any relevant domain)
- **📝 Product Details**: A short, single-line description of the product
- **💰 Tentative Price**: Estimated price in **USD** (as an integer)

In [1]:
# ✅ Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  # Set GROQ API Key

# ✅ Initialize Groq LLM (LLaMA3 model)
from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-70b-8192")

# ✅ Define the output schema using Pydantic
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class Product(BaseModel):
    name : str = Field(description="This is the name of the product from textile company which should be one word")
    details: str = Field(description="Give the details of the product in 1 line")
    price: int = Field(description="price should be in USD")

# ✅ Set up output parser
parser = JsonOutputParser(pydantic_object=Product)

# ✅ Create prompt with instructions and format
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(
    template=(
        "You are an expert assistant that always replies with only valid JSON, nothing else.\n"
        "{format_instructions}\n"
        "Query: {query}"
    ),
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# ✅ Build the chain: Prompt → LLM → Parser
chain = prompt | model | parser

# ✅ Invoke the chain with a query
response = chain.invoke({"query": "Tell me about the Burton's snowsuit"})

# ✅ Print the structured result
print(response)


{'name': 'Burtons', 'details': 'Waterproof snowsuit for extreme weather', 'price': 250}
